In [1]:
#import
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
#데이터 가져오기
data = pd.read_csv("/content/drive/MyDrive/공부(개인)/인공지능/컴패스/data/data_set.csv")
print(data)

         USABILITY            BLD_NM
0          22000.0  연기군 생활폐기물 종합처리시설
1           1000.0                2동
2          17000.0            문곡리 공장
3           1000.0                1동
4          10000.0         세종하이텍고등학교
...            ...               ...
1960869   100000.0             공주대학교
1960870   100000.0                3동
1960871   100000.0                2동
1960872   100000.0                7동
1960873   100000.0               13동

[1960874 rows x 2 columns]


In [3]:
#데이터 수정
data['BLD_NM'] = data['BLD_NM'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9]','')
print(data)

<ipython-input-3-b2558f775ddf>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data['BLD_NM'] = data['BLD_NM'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9]','')


         USABILITY          BLD_NM
0          22000.0  연기군생활폐기물종합처리시설
1           1000.0              2동
2          17000.0           문곡리공장
3           1000.0              1동
4          10000.0       세종하이텍고등학교
...            ...             ...
1960869   100000.0           공주대학교
1960870   100000.0              3동
1960871   100000.0              2동
1960872   100000.0              7동
1960873   100000.0             13동

[1960874 rows x 2 columns]


In [4]:
print(data.isnull().sum()) #빈칸 확인
print(data.drop_duplicates(subset=['USABILITY'],inplace=False) )
print(len(data.drop_duplicates(subset=['USABILITY'],inplace=False) ))

USABILITY    18
BLD_NM        0
dtype: int64
         USABILITY                     BLD_NM
0          22000.0             연기군생활폐기물종합처리시설
1           1000.0                         2동
2          17000.0                      문곡리공장
4          10000.0                  세종하이텍고등학교
8          11000.0               사회복지법인동면어린이집
...            ...                        ...
1950885   270000.0  어진동6관광휴게시설한국토지주택공사세종사업본부장
1950915   160000.0                 이석호씨정리상가신축
1950966    70000.0          조치원읍원리1522위락시설김종원
1950972    80000.0                   세종여자고등학교
1956026   310000.0                     오시유야영장

[77 rows x 2 columns]
77


In [5]:
#전처리 bag of words
utext = data['BLD_NM'].tolist()
utext = ''.join(utext)
utext = list(set(utext))
utext.sort()
print(utext[0:100])

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅎ', 'ㅡ', 'ㅣ', '가', '각', '간', '갈', '감', '갑', '값', '갓', '강', '같', '개', '객', '갤', '갯', '갱', '갹', '걀', '거', '건', '걸', '검', '것', '겅', '게', '겐', '겔', '겜', '겨', '격', '견', '결', '겸', '겹', '경', '계', '곌', '곕', '곗', '고', '곡', '곤', '곧', '골', '곰', '곱', '곳', '공', '곶', '과', '곽', '관', '괄', '광', '괘', '괴', '굄', '굉', '교', '굔', '구', '국', '군', '굳', '굴', '굼', '굽', '굿', '궁', '궈', '궉', '권', '궐', '궤', '귀', '규', '균', '귤', '그']


In [6]:
#Tokenizer 문자를 숫자로 바꾸기

tokenizer = Tokenizer(char_level=True,oov_token='<OOV>') #글자단위로 정수로 변환 False는 단어 oov 새로운 단어 처리
text_list = data['BLD_NM'].tolist()
tokenizer.fit_on_texts(text_list) #문자리스트를 넣음

print(tokenizer.word_index)
print(text_list[0:10]) #사용할 문자 데이터

seq = tokenizer.texts_to_sequences(text_list) #문자를 정수로 바꾸기
print(seq[0:10])

{'<OOV>': 1, '동': 2, '주': 3, '아': 4, '빌': 5, '트': 6, '파': 7, '1': 8, '택': 9, '리': 10, '라': 11, '2': 12, '대': 13, '이': 14, '스': 15, '교': 16, '성': 17, '단': 18, '지': 19, '공': 20, '시': 21, '산': 22, '원': 23, '장': 24, '학': 25, '가': 26, '사': 27, '마': 28, '신': 29, '제': 30, '3': 31, '한': 32, '독': 33, '호': 34, '우': 35, '하': 36, '0': 37, '을': 38, '고': 39, '설': 40, '린': 41, '영': 42, '정': 43, '회': 44, '5': 45, '화': 46, '4': 47, '운': 48, '수': 49, '타': 50, '차': 51, '관': 52, '진': 53, '등': 54, '소': 55, '현': 56, '서': 57, '딩': 58, '구': 59, '창': 60, '삼': 61, '상': 62, '6': 63, '양': 64, '다': 65, '자': 66, '남': 67, '인': 68, '경': 69, '씨': 70, '8': 71, '생': 72, '7': 73, '천': 74, '근': 75, '면': 76, '연': 77, '초': 78, '종': 79, '9': 80, '중': 81, '미': 82, '세': 83, '부': 84, '림': 85, '오': 86, '유': 87, '일': 88, '도': 89, '전': 90, '축': 91, '물': 92, '크': 93, '나': 94, '안': 95, '로': 96, '기': 97, '광': 98, '청': 99, '건': 100, '금': 101, '업': 102, '그': 103, '김': 104, '강': 105, '용': 106, '활': 107, '에': 108, '보': 109, '식': 110, '명'

In [7]:
Y = data['USABILITY'].tolist() # Y 데이터
#Y = data.pop('USABILITY')
print(Y[0:30])

[22000.0, 1000.0, 17000.0, 1000.0, 10000.0, 10000.0, 10000.0, 17000.0, 11000.0, 1000.0, 1000.0, 17000.0, 21000.0, 10000.0, 17000.0, 17000.0, 17000.0, 26000.0, 13000.0, 21000.0, 21000.0, 1000.0, 2000.0, 21000.0, 22000.0, 1000.0, 1000.0, 18000.0, 18000.0, 4000.0]


In [8]:
#학습 데이터
X = pad_sequences(seq)
trainX,valX,trainY,valY=train_test_split(X,Y,test_size=0.1, random_state=42)


1764786
196088


In [14]:
print(len(trainX))
print(len(trainY))
print(len(valX))
print(len(valY))

1764786
1764786
196088
196088
1960874


In [10]:
#모델 만들기
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(len(utext)+1,16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(???, activation='softmax')
])

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 16)          19744     
                                                                 
 lstm_1 (LSTM)               (None, 64)                20736     
                                                                 
 dense_1 (Dense)             (None, 1960874)           127456810 
                                                                 
Total params: 127,497,290
Trainable params: 127,497,290
Non-trainable params: 0
_________________________________________________________________


In [11]:
trainX = np.array(trainX)
trainY = np.array(trainY)

In [13]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(trainX, trainY,batch_size=32, epochs=5)

Epoch 1/5
  876/55150 [..............................] - ETA: 17:28 - loss: -234636.0156 - acc: 0.0000e+00

KeyboardInterrupt: ignored